# Fitting CHIME to NYC Data

In [1]:
from os import environ
from datetime import date, timedelta

from pandas import DataFrame, date_range
from numpy import array, linspace, log, sqrt, exp, arange

import plotly.graph_objects as go
from plotly.subplots import make_subplots

from gvar import gvar, mean, sdev, fmt_values, fmt_errorbudget
from lsqfit import nonlinear_fit

from models import sir_step, sihr_step, one_minus_logistic_fcn, FitFcn
from prepare_df import prepare_case_hosp_death, COMMIT_HASH_LAST
import plotting
from importlib import reload

In [2]:
print(COMMIT_HASH_LAST)
#!pip show penn_chime

e19db289166f73282d39dfcef0d47a324d654c07


In [271]:
BIN_SIZE = 1

chd_df = prepare_case_hosp_death(
    COMMIT_HASH_LAST, bin_day_range=BIN_SIZE, drop_days_end=8
).loc["2020-03-08":"2020-04-01"]
chd_df.tail()

,death_cummulative,death_new,hospitalized_cummulative,hospitalized_new,infected_cummulative,infected_new
date,,,,,,
2020-03-28,1052,230,10800,1098,44582,2851
2020-03-29,1299,247,11984,1184,47538,2956
2020-03-30,1565,266,13368,1384,52437,4899
2020-03-31,1884,319,14594,1226,56423,3986
2020-04-01,2221,337,15751,1157,60512,4089


In [272]:
X = {
    "initial_susceptible": int(1.8e6),
    "initial_hospitalized": chd_df.hospitalized_cummulative.iloc[0],
    "initial_recovered": 0,
    "n_iter": chd_df.shape[0] + 1,
}
print(X)


infected_new = chd_df["infected_new"].values
hospitalized_new = chd_df["hospitalized_new"].values

infected_new *= 1

delta_infected_new = [max(100, infected * 0.5) for infected in infected_new]
delta_hospitalized_new = [
    max(10, hospitalized * 0.05) for hospitalized in hospitalized_new
]

Y = gvar(
    [infected_new, hospitalized_new], [delta_infected_new, delta_hospitalized_new],
).T

{'initial_susceptible': 1800000, 'initial_hospitalized': 73, 'initial_recovered': 0, 'n_iter': 26}


# Fits

## Regular SIR, constant beta

In [99]:
beta0 = log(2) / 2 / X["initial_susceptible"]
beta0

1.925408834888737e-07

In [100]:
fcn = FitFcn(
    sir_step, columns=["infected_new", "hospitalized_new"], as_array=True, drop_rows=[0]
)
prior = {
    "beta_i": gvar(beta0, beta0 * 0.8),
    "gamma_i": 1 / (gvar(14, 5) / BIN_SIZE),
    "initial_infected": gvar(1.0e4, 2.0e4),
    "hospitalization_rate": gvar(0.05, 0.1),
}

In [101]:
fit = nonlinear_fit(data=(X, Y), fcn=fcn, prior=prior)
print(fit)

fit_res = FitFcn(sir_step, drop_rows=[0])(X, fit.p)

outputs = dict(**fit.p)
outputs.update(
    {
        "doubling_time": log(2) / (fit.p["beta_i"] * X["initial_susceptible"]),
        "recovery in days": 1 / fit.p["gamma_i"] * BIN_SIZE,
    }
)
inputs = dict(y=Y, **prior)

outputs = {
    "".join([kk[0] for kk in key.split("_")]): val for key, val in outputs.items()
}

print(fmt_values(outputs))
print(fmt_errorbudget(outputs, inputs))

Least Square Fit:
  chi2/dof [dof] = 17 [50]    Q = 4e-147    logGBF = -735.48

Parameters:
              beta_i   1.31(14)e-07      [  1.9(1.5)e-07 ]  
             gamma_i     0.095 (25)      [    0.071 (26) ]  
    initial_infected     1075 (159)      [ 10000 (20000) ]  
hospitalization_rate     0.321 (32)      [     0.05 (10) ]  **

Settings:
  svdcut/n = 1e-12/0    tol = (1e-08,1e-10,1e-10*)    (itns/time = 17/0.1)
  fitter = scipy_least_squares    method = trf

Values:
                 bi: 1.31(14)e-07        
                 gi: 0.095(25)           
                 ii: 1075(159)           
                 hr: 0.321(32)           
                 dt: 2.95(32)            
                  r: 10.5(2.8)           

Partial % Errors:
                             bi        gi        ii        hr        dt         r
---------------------------------------------------------------------------------
                   y:      0.87      0.88      9.44      9.37      0.87      0.88
   

In [102]:
reload(plotting)

<module 'plotting' from '/Users/christopherkorber/gdrive/git/bochum/covid-19-analysis/plotting.py'>

In [103]:
fig = make_subplots(
    rows=1,
    cols=2,
    subplot_titles=("New infections", "New hospitalizations", "Plot 3", "Plot 4"),
)
plotting.add_gvar_scatter(
    fig, x=chd_df.index, y=Y.T[0], name="Data", mode="markers+lines", color="#1f77b4",
)
plotting.add_gvar_scatter(
    fig,
    x=chd_df.index,
    y=fit_res.infected_new.values,
    name="Fit",
    mode="markers+lines",
    gv_mode="band",
    color="#bcbd22",
)
plotting.add_gvar_scatter(
    fig,
    x=chd_df.index,
    y=Y.T[1],
    name="Data",
    mode="markers+lines",
    row=1,
    col=2,
    color="#1f77b4",
    showlegend=False,
)
plotting.add_gvar_scatter(
    fig,
    x=chd_df.index,
    y=fit_res.hospitalized_new.values,
    name="Fit",
    mode="markers+lines",
    gv_mode="band",
    color="#bcbd22",
    row=1,
    col=2,
    showlegend=False,
)
fig.update_layout(
    # yaxis1={"type":"log"},
    # yaxis2={"type": "log"}
)
fig.show()

## Now with social distancing

In [157]:
fcn = FitFcn(
    sir_step,
    columns=["infected_new", "hospitalized_new"],
    beta_i_fcn=one_minus_logistic_fcn,
    as_array=True,
    drop_rows=[0],
)
prior = {
    "beta_i": gvar(beta0, beta0 * 0.8),
    "gamma_i": 1 / (gvar(14, 5) / BIN_SIZE),
    "initial_infected": gvar(1.0e3, 2.0e3),
    "beta_h": gvar(0.05, 0.1),
    "gamma_h": 1 / (gvar(14, 5) / BIN_SIZE),
    "ratio": gvar(0.7, 0.3),
    "decay_width": 1 / gvar(4, 1),
    "x0": gvar(9, 2),
    "hospitalization_rate": gvar(0.1, 0.2),
}

In [158]:
fit = nonlinear_fit(data=(X, Y), fcn=fcn, prior=prior)
print(fit)


n_days = 100
X_extra = X.copy()
X_extra["n_iter"] = n_days
days_extra = date_range(chd_df.index.min(), periods=n_days - 1, freq="D") + timedelta(
    days=1
)

fit_res = FitFcn(sir_step, drop_rows=[0], beta_i_fcn=one_minus_logistic_fcn)(
    X_extra, fit.p
)
fit_res.index = days_extra


outputs = dict(fit.p)
outputs.update(
    {
        "doubling_time": log(2) / (fit.p["beta_i"] * X["initial_susceptible"]),
        "recovery_in_days": 1 / fit.p["gamma_i"] * BIN_SIZE,
    }
)
inputs = dict(y=Y, **prior)

outputs = {
    "".join([kk[0] if len(key.split("_")) > 1 else kk for kk in key.split("_")]): val
    for key, val in outputs.items()
}


print(fmt_values(outputs))
print(fmt_errorbudget(outputs, inputs))

Least Square Fit:
  chi2/dof [dof] = 1.4 [50]    Q = 0.025    logGBF = -347.33

Parameters:
              beta_i   3.72(52)e-07      [ 1.9(1.5)e-07 ]  *
             gamma_i     0.069 (25)      [   0.071 (26) ]  
    initial_infected       136 (26)      [  1000 (2000) ]  
              beta_h      0.05 (10)      [    0.05 (10) ]  
             gamma_h     0.071 (26)      [   0.071 (26) ]  
               ratio     0.914 (56)      [    0.70 (30) ]  
         decay_width     0.142 (26)      [   0.250 (62) ]  *
                  x0      9.9 (1.7)      [    9.0 (2.0) ]  
hospitalization_rate     0.285 (32)      [    0.10 (20) ]  

Settings:
  svdcut/n = 1e-12/0    tol = (1e-08,1e-10,1e-10*)    (itns/time = 55/0.3)
  fitter = scipy_least_squares    method = trf

Values:
                 bi: 3.72(52)e-07        
                 gi: 0.069(25)           
                 ii: 136(26)             
                 bh: 0.05(10)            
                 gh: 0.071(26)           
              

In [159]:
fig = make_subplots(
    rows=1,
    cols=2,
    subplot_titles=("New infections", "New hospitalizations", "Plot 3", "Plot 4"),
)
plotting.add_gvar_scatter(
    fig, x=chd_df.index, y=Y.T[0], name="Data", mode="markers+lines", color="#1f77b4",
)
plotting.add_gvar_scatter(
    fig,
    x=fit_res.index,
    y=fit_res.infected_new.values,
    name="Fit",
    mode="markers+lines",
    gv_mode="band",
    color="#bcbd22",
)
plotting.add_gvar_scatter(
    fig,
    x=chd_df.index,
    y=Y.T[1],
    name="Data",
    mode="markers+lines",
    row=1,
    col=2,
    color="#1f77b4",
    showlegend=False,
)
plotting.add_gvar_scatter(
    fig,
    x=fit_res.index,
    y=fit_res.hospitalized_new.values,
    name="Fit",
    mode="markers+lines",
    gv_mode="band",
    color="#bcbd22",
    row=1,
    col=2,
    showlegend=False,
)
fig.update_layout(
    # yaxis1={"type":"log"},
    # yaxis2={"type": "log"}
)
fig.show()

In [160]:
reload(plotting)

<module 'plotting' from '/Users/christopherkorber/gdrive/git/bochum/covid-19-analysis/plotting.py'>

In [161]:
beta = fcn.beta_i_fcn(arange(X_extra["n_iter"]), amplitude=fit.p["beta_i"], **fit.p)
dt = log(2) / (beta[1:] * fit_res.susceptible.values)

fig = go.Figure()
plotting.add_gvar_scatter(
    fig,
    x=chd_df.index,
    y=dt,
    name="Fit",
    mode="markers+lines",
    gv_mode="band",
    color="#bcbd22",
)
fig.update_layout(title="Effective Doubling Time R0", yaxis_title="Days")
fig.show()

In [165]:
fit_res.infected.values

array([201(35), 294(48), 427(64), 612(87), 866(117), 1209(158), 1666(213),
       2259(286), 3017(381), 3964(503), 5121(656), 6503(844), 8118(1068),
       9962(1332), 12019(1635), 14262(1978), 16655(2359), 19153(2776),
       21705(3226), 24261(3705), 26771(4209), 29190(4733), 31481(5272),
       33611(5823), 35560(6382), 37312(6948), 38860(7523), 40204(8106),
       41347(8701), 42298(9312), 43069(9941), 43671(10590), 44119(11263),
       44428(11959), 44612(12678), 44684(13418), 44658(14178),
       44545(14954), 44357(15743), 44105(16541), 43796(17343),
       43440(18147), 43043(18949), 42612(19744), 42152(20531),
       41669(21306), 41166(22066), 40648(22810), 40117(23535),
       39577(24241), 39029(24926), 38476(25588), 37920(26227),
       37362(26842), 36804(27432), 36246(27998), 35690(28540),
       35136(29056), 34586(29547), 34039(30013), 33496(30455),
       32958(30872), 32425(31265), 31897(31634), 31375(31980),
       30859(32303), 30348(32603), 29844(32882), 29345(331

In [166]:
fig = go.Figure()
plotting.add_gvar_scatter(
    fig,
    x=fit_res.index,
    y=fit_res.infected.values,
    name="Fit",
    mode="markers+lines",
    gv_mode="band",
    color="#bcbd22",
)
fig.update_layout(title="Effective Doubling Time R0", yaxis_title="Days")
fig.show()

## SIHR model without distancing

In [109]:
fcn = FitFcn(
    sihr_step,
    columns=["infected_new", "hospitalized_new"],
    as_array=True,
    drop_rows=[0],
)
prior = {
    "beta_i": gvar(beta0, beta0 * 0.8),
    "gamma_i": 1 / (gvar(14, 5) / BIN_SIZE),
    "initial_infected": gvar(1.0e4, 2.0e4),
    "beta_h": gvar(0.05, 0.1),
    "gamma_h": 1 / (gvar(14, 5) / BIN_SIZE),
}

Y_SIHR = Y.copy()
Y_SIHR.T[0] -= Y_SIHR.T[1]

X_SIHR = X.copy()
X_SIHR["capacity"] = 20000
X_SIHR

{'initial_susceptible': 1800000,
 'initial_hospitalized': 73,
 'initial_recovered': 0,
 'n_iter': 26,
 'capacity': 20000}

In [110]:
fit = nonlinear_fit(data=(X_SIHR, Y_SIHR), fcn=fcn, prior=prior)
print(fit)

X_SIHR_extra = X_SIHR.copy()
X_SIHR_extra["n_iter"] = 100
days_extra = date_range(chd_df.index.min(), periods=100 - 1, freq="D")

fit_res = FitFcn(sihr_step, drop_rows=[0])(X_SIHR_extra, fit.p)
fit_res.index = days_extra


outputs = dict(**fit.p)
outputs.update(
    {
        "doubling_time": log(2) / (fit.p["beta_i"] * X["initial_susceptible"]),
        "recovery in days": 1 / fit.p["gamma_i"] * BIN_SIZE,
    }
)
inputs = dict(y=Y, **prior)

outputs = {
    "".join([kk[0] for kk in key.split("_")]): val for key, val in outputs.items()
}

print(fmt_values(outputs))
print(fmt_errorbudget(outputs, inputs))

Least Square Fit:
  chi2/dof [dof] = 17 [50]    Q = 2.7e-147    logGBF = -736.39

Parameters:
          beta_i   2.61(42)e-07      [  1.9(1.5)e-07 ]  
         gamma_i     0.084 (25)      [    0.071 (26) ]  
initial_infected       332 (88)      [ 10000 (20000) ]  
          beta_h     0.246 (66)      [     0.05 (10) ]  *
         gamma_h     0.071 (26)      [    0.071 (26) ]  

Settings:
  svdcut/n = 1e-12/0    tol = (1e-08,1e-10,1e-10*)    (itns/time = 30/0.3)
  fitter = scipy_least_squares    method = trf

Values:
                 bi: 2.61(42)e-07        
                 gi: 0.084(25)           
                 ii: 332(88)             
                 bh: 0.246(66)           
                 gh: 0.071(26)           
                 dt: 1.48(23)            
                  r: 11.9(3.5)           

Partial % Errors:
                         bi        gi        ii        bh        gh        dt         r
-----------------------------------------------------------------------------

In [111]:
fig = make_subplots(
    rows=1,
    cols=2,
    subplot_titles=("New infections", "New hospitalizations", "Plot 3", "Plot 4"),
)
plotting.add_gvar_scatter(
    fig,
    x=chd_df.index,
    y=Y_SIHR.T[0],
    name="Data",
    mode="markers+lines",
    color="#1f77b4",
)
plotting.add_gvar_scatter(
    fig,
    x=fit_res.index,
    y=fit_res.infected_new.values,
    name="Fit",
    mode="markers+lines",
    gv_mode="band",
    color="#bcbd22",
)
plotting.add_gvar_scatter(
    fig,
    x=chd_df.index,
    y=Y_SIHR.T[1],
    name="Data",
    mode="markers+lines",
    row=1,
    col=2,
    color="#1f77b4",
    showlegend=False,
)
plotting.add_gvar_scatter(
    fig,
    x=fit_res.index,
    y=fit_res.hospitalized_new.values,
    name="Fit",
    mode="markers+lines",
    gv_mode="band",
    color="#bcbd22",
    row=1,
    col=2,
    showlegend=False,
)
fig.update_layout(
    # yaxis1={"type":"log"},
    # yaxis2={"type": "log"}
)
fig.show()

## SIHR with distancing

In [112]:
fcn = FitFcn(
    sihr_step,
    columns=["infected_new", "hospitalized_new"],
    as_array=True,
    drop_rows=[0],
    beta_i_fcn=one_minus_logistic_fcn,
)
prior = {
    "beta_i": gvar(beta0, beta0 * 0.8),
    "gamma_i": 1 / (gvar(14, 5) / BIN_SIZE),
    "initial_infected": gvar(1.0e3, 2.0e3),
    "beta_h": gvar(0.05, 0.1),
    "gamma_h": 1 / (gvar(14, 5) / BIN_SIZE),
    "ratio": gvar(0.7, 0.3),
    "decay_width": 1 / gvar(4, 1),
    "x0": gvar(9, 2),
}

X_SIHR = X.copy()
X_SIHR["capacity"] = 20000
X_SIHR

{'initial_susceptible': 1800000,
 'initial_hospitalized': 73,
 'initial_recovered': 0,
 'n_iter': 26,
 'capacity': 20000}

In [113]:
fit = nonlinear_fit(data=(X_SIHR, Y_SIHR), fcn=fcn, prior=prior)
print(fit)

X_SIHR_extra = X_SIHR.copy()
X_SIHR_extra["n_iter"] = 100
days_extra = date_range(chd_df.index.min(), periods=100 - 1, freq="D")

fit_res = FitFcn(sihr_step, drop_rows=[0], beta_i_fcn=one_minus_logistic_fcn,)(
    X_SIHR_extra, fit.p
)
fit_res.index = days_extra

outputs = dict(**fit.p)
outputs.update(
    {
        "doubling_time": log(2) / (fit.p["beta_i"] * X["initial_susceptible"]),
        "recovery in days": 1 / fit.p["gamma_i"] * BIN_SIZE,
    }
)
inputs = dict(y=Y, **prior)

outputs = {
    "".join([kk[0] for kk in key.split("_")]): val for key, val in outputs.items()
}

print(fmt_values(outputs))
print(fmt_errorbudget(outputs, inputs))

Least Square Fit:
  chi2/dof [dof] = 1.5 [50]    Q = 0.016    logGBF = -347.18

Parameters:
          beta_i   4.08(53)e-07      [ 1.9(1.5)e-07 ]  *
         gamma_i     0.074 (24)      [   0.071 (26) ]  
initial_infected       138 (35)      [  1000 (2000) ]  
          beta_h     0.157 (38)      [    0.05 (10) ]  *
         gamma_h     0.071 (26)      [   0.071 (26) ]  
           ratio     0.747 (86)      [    0.70 (30) ]  
     decay_width     0.186 (42)      [   0.250 (62) ]  *
              x0      9.5 (1.5)      [    9.0 (2.0) ]  

Settings:
  svdcut/n = 1e-12/0    tol = (1e-08,1e-10,1e-10*)    (itns/time = 38/0.3)
  fitter = scipy_least_squares    method = trf

Values:
                 bi: 4.08(53)e-07        
                 gi: 0.074(24)           
                 ii: 138(35)             
                 bh: 0.157(38)           
                 gh: 0.071(26)           
                  r: 13.6(4.5)           
                 dw: 0.186(42)           
                  x: 

In [114]:
fig = make_subplots(
    rows=1,
    cols=2,
    subplot_titles=(
        "New infections - hospitalizations",
        "New hospitalizations",
        "Plot 3",
        "Plot 4",
    ),
)
plotting.add_gvar_scatter(
    fig,
    x=chd_df.index,
    y=Y_SIHR.T[0],
    name="Data",
    mode="markers+lines",
    color="#1f77b4",
)
plotting.add_gvar_scatter(
    fig,
    x=fit_res.index,
    y=fit_res.infected_new.values,
    name="Fit",
    mode="markers+lines",
    gv_mode="band",
    color="#bcbd22",
)
plotting.add_gvar_scatter(
    fig,
    x=chd_df.index,
    y=Y_SIHR.T[1],
    name="Data",
    mode="markers+lines",
    row=1,
    col=2,
    color="#1f77b4",
    showlegend=False,
)
plotting.add_gvar_scatter(
    fig,
    x=fit_res.index,
    y=fit_res.hospitalized_new.values,
    name="Fit",
    mode="markers+lines",
    gv_mode="band",
    color="#bcbd22",
    row=1,
    col=2,
    showlegend=False,
)
fig.update_layout(
    # yaxis1={"type":"log"},
    # yaxis2={"type": "log"}
)
fig.show()

In [115]:
beta = fcn.beta_i_fcn(
    arange(X_SIHR_extra["n_iter"]), amplitude=fit.p["beta_i"], **fit.p
)
dt = log(2) / (beta[1:] * fit_res.susceptible.values)

fig = go.Figure()
plotting.add_gvar_scatter(
    fig,
    x=chd_df.index,
    y=dt,
    name="Fit",
    mode="markers+lines",
    gv_mode="band",
    color="#bcbd22",
)
fig.update_layout(title="Effective Doubling Time R0", yaxis_title="Days")
fig.show()

In [116]:
fit_res

,susceptible,infected,hospitalized,recovered,infected_new,hospitalized_new,recovered_new
2020-03-08,1.799910(17)e+06,197(49),89.4(3.1),153(37),90(17),21.6(2.4),15.4(4.2)
2020-03-09,1.799784(39)e+06,277(68),113.8(6.4),217(52),126(22),30.8(2.8),20.8(5.7)
2020-03-10,1.799610(69)e+06,387(94),149(10),306(72),174(30),43.4(3.1),28.5(7.8)
2020-03-11,1.79937(11)e+06,534(129),199(15),426(99),236(41),60.6(3.5),39(11)
2020-03-12,1.79906(16)e+06,725(175),268(20),587(135),314(54),83.6(3.7),53(15)
...,...,...,...,...,...,...,...
2020-06-10,1.737(24)e+06,256(604),1559(2448),406(832),48(120),42(97),136(209)
2020-06-11,1.737(24)e+06,243(582),1488(2374),386(802),46(115),40(94),130(203)
2020-06-12,1.737(24)e+06,231(560),1419(2302),367(774),44(111),38(90),124(196)
2020-06-13,1.737(24)e+06,219(539),1354(2231),349(746),41(107),36(87),118(191)


In [117]:
fig = go.Figure()
plotting.add_gvar_scatter(
    fig,
    x=fit_res.index,
    y=fit_res.hospitalized.values,
    name="Fit",
    mode="markers+lines",
    gv_mode="band",
    color="#bcbd22",
)
fig.update_layout(title="Total number of hospitalizations", yaxis_title="Days")
fig.show()

## SIHR with distancing without infected

In [279]:
fcn = FitFcn(
    sihr_step,
    columns=["hospitalized_new"],
    as_array=True,
    drop_rows=[0],
    beta_i_fcn=one_minus_logistic_fcn,
)
prior = {
    "beta_i": gvar(beta0, beta0 * 0.8),
    "gamma_i": 1 / (gvar(14, 9) / BIN_SIZE),
    "initial_infected": gvar(1.0e3, 2.0e3),
    "beta_h": gvar(0.05, 0.1),
    "gamma_h": 1 / (gvar(14, 5) / BIN_SIZE),
    "ratio": gvar(0.7, 0.3),
    "decay_width": 1 / gvar(4, 1),
    "x0": gvar(9, 2),
}

X_SIHR = X.copy()
X_SIHR["capacity"] = 20000
Y_SIHR_H = Y.T[1].copy()

In [280]:
fit = nonlinear_fit(data=(X_SIHR, Y_SIHR_H), fcn=fcn, prior=prior)
print(fit)

X_SIHR_extra = X_SIHR.copy()
X_SIHR_extra["n_iter"] = 100
days_extra = date_range(chd_df.index.min(), periods=100 - 1, freq="D")

fit_res = FitFcn(sihr_step, drop_rows=[0], beta_i_fcn=one_minus_logistic_fcn,)(
    X_SIHR_extra, fit.p
)
fit_res.index = days_extra

outputs = dict(**fit.p)
outputs.update(
    {
        "doubling_time": log(2) / (fit.p["beta_i"] * X["initial_susceptible"]),
        "recovery in days": 1 / fit.p["gamma_i"] * BIN_SIZE,
    }
)
inputs = dict(y=Y, **prior)

outputs = {
    "".join([kk[0] for kk in key.split("_")]): val for key, val in outputs.items()
}

print(fmt_values(outputs))
print(fmt_errorbudget(outputs, inputs))

Least Square Fit:
  chi2/dof [dof] = 2.3 [25]    Q = 0.00016    logGBF = -147.14

Parameters:
          beta_i   3.41(65)e-07      [ 1.9(1.5)e-07 ]  
         gamma_i     0.072 (44)      [   0.071 (46) ]  
initial_infected      500 (870)      [  1000 (2000) ]  
          beta_h     0.045 (78)      [    0.05 (10) ]  
         gamma_h     0.071 (26)      [   0.071 (26) ]  
           ratio      0.89 (14)      [    0.70 (30) ]  
     decay_width     0.180 (43)      [   0.250 (62) ]  *
              x0      9.7 (1.5)      [    9.0 (2.0) ]  

Settings:
  svdcut/n = 1e-12/0    tol = (1e-08,1e-10,1e-10*)    (itns/time = 66/0.3)
  fitter = scipy_least_squares    method = trf

Values:
                 bi: 3.41(65)e-07        
                 gi: 0.072(44)           
                 ii: 500(870)            
                 bh: 0.045(78)           
                 gh: 0.071(26)           
                  r: 14.0(8.7)           
                 dw: 0.180(43)           
                  x: 

In [281]:
fig = make_subplots(
    rows=1,
    cols=2,
    subplot_titles=(
        "New hospitalizations",
        "New Infections (Prediction)",
        "Plot 3",
        "Plot 4",
    ),
)
plotting.add_gvar_scatter(
    fig,
    x=chd_df.index,
    y=Y_SIHR_H,
    name="Data",
    mode="markers+lines",
    row=1,
    col=1,
    color="#1f77b4",
    showlegend=False,
)
plotting.add_gvar_scatter(
    fig,
    x=fit_res.index,
    y=fit_res.hospitalized_new.values,
    name="Fit",
    mode="markers+lines",
    gv_mode="band",
    color="#bcbd22",
    row=1,
    col=1,
)
fig.add_trace(
    go.Scatter(
        x=chd_df.index,
        y=chd_df.infected_new,
        name="Data",
        mode="markers+lines",
        line_color="#1f77b4",
    ),
    row=1,
    col=2,
)
plotting.add_gvar_scatter(
    fig,
    x=fit_res.index,
    y=fit_res.infected_new.values + fit_res.hospitalized_new.values,
    name="Prediction",
    mode="markers+lines",
    gv_mode="band",
    color="#bcbd22",
    row=1,
    col=2,
    showlegend=False,
)
fig.update_layout(
    #yaxis1={"type":"log"},
    #yaxis2={"type": "log"}
)
fig.show()

In [276]:
fig = go.Figure()
plotting.add_gvar_scatter(
    fig,
    x=fit_res.index,
    y=fit_res.hospitalized.values,
    name="Fit",
    mode="markers+lines",
    gv_mode="band",
    color="#bcbd22",
)
fig.update_layout(title="Total number of hospitalizations", yaxis_title="Days")
fig.show()

In [277]:
fig = go.Figure()
plotting.add_gvar_scatter(
    fig,
    x=fit_res.index,
    y=fit_res.infected.values + fit_res.hospitalized.values,
    name="Fit",
    mode="markers+lines",
    gv_mode="band",
    color="#bcbd22",
)
fig.update_layout(title="Total number of infections", yaxis_title="Days")
fig.show()

In [282]:
beta = fcn.beta_i_fcn(
    arange(X_SIHR_extra["n_iter"]), amplitude=fit.p["beta_i"], **fit.p
)
dt = log(2) / (beta[1:] * fit_res.susceptible.values)

fig = go.Figure()
plotting.add_gvar_scatter(
    fig,
    x=fit_res.index,
    y=dt,
    name="Fit",
    mode="markers+lines",
    gv_mode="band",
    color="#bcbd22",
)
fig.update_layout(title="Effective Doubling Time R0", yaxis_title="Days")
fig.show()